In [ ]:
!pip install llama-index
!pip install langchain

In [ ]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, VectorStoreIndex, LLMPredictor, PromptHelper, ServiceContext,load_index_from_storage, StorageContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 0.3
    # set chunk size limit
    chunk_size_limit = 600

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))

    documents = SimpleDirectoryReader(directory_path).load_data()

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)

    # index.save_to_disk('index.json')
    index.storage_context.persist()

    return index

def ask_ai():
    storage_context = StorageContext.from_defaults(persist_dir="./storage/")
    index = load_index_from_storage(storage_context)
    while True:
        query = input("What do you want to ask? ")
        query_engine = index.as_query_engine()
        response = query_engine.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))


In [ ]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

In [ ]:
construct_index("data")

In [ ]:
ask_ai()